In [ ]:
# Referenced 10.3 #10 Flask with ORM--utilized this activity heavily and rewatched example multiple times

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [ ]:
# Database Setup
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# Reflect an existing database into a new model
Base = automap_base()

# Reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
Station = Base.classes.station
Measurement = Base.classes.measurement

In [ ]:
# Create session link to database
session = Session(engine)

In [ ]:
# Flask Set Up
# Create an app being sure to pass __name__
app = Flask(__name__)

In [ ]:
# Flask Routes
@app.route("/")
def welcome():
    """List all available API routes"""
    return (
       f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/min_avg_max/<start><br/>"
        f"/api/v1.0/min_avg_max/<start>/<end><br/>"
    )

# Precipitation Route
@app.route("/api/v1.0/precipitation")
def precipitation():
    print("Request for 'Precipication' page...")
    
    # Create session link to DB
    session= Session(engine)
    
    # return a list of all Precipitation dates through a query
    # results = session.query(Measurement.date, Measurement.prcp).all()
    
    # Calculate the date 1 year ago from last data point in database
    # Referenced activity 10.3 #2 Reviewing of DateTime (calcualating a time delta and printing date objects)
    ly_date = dt.date(2017,8,23) - dt.timedelta(days=365)

    # Query for the date and precipitation for the last year
    ly_data = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= ly_date).all()
    
    # Close Session
    session.close()
    
    # Convert list of tuples into normal list
    # Reference 10.3 #3 SQLAlchemy, SQLite, and Dates
    precipitation = list(np.ravel(results))
    
    # Convert list to dictionary
    
    # jsonify results
    return jsonify(precipitations)

In [ ]:
# Stations Route
@app.route("/api/v1.0/stations")
def stations():
    print("Request for 'Stations' page...")
    
    # Create session link to DB
    session= Session(engine)
    
    # Return a list of all Precipitation dates through a query
    results = session.query(Station.station).all()
    
    # Close Session
    session.close()
    
    # Convert list of tuples into normal list
    # Reference 10.3 #10 and #11
    stations = list(np.ravel(results))
   
    # jsonify results
    return jsonify(stations=stations)

In [ ]:
# TOBS Route
@app.route("/api/v1.0/tobs")
def temperature_observations():
    print("Request for 'TOBS' page...")
    
    # Create session link to DB
    session= Session(engine)
    
    # Referenced activity 10.3 #2 Reviewing of DateTime (calcualating a time delta and printing date objects)
    ly_date = dt.date(2017,8,23) - dt.timedelta(days=365)
    
    # Query dates and tobs of most active station for ly data
    # Referenced activity 10.3 #1 Filtering Review
    results = session.query(Measurement.tobs).\
        filter(Measurement.station == 'USC00519281').\
        filter(Measurement.date >= ly_date).all()

    # Close Session
    session.close()
    
    # Convert list of tuples into normal list
    # Reference 10.3 #10 and #11
    ly_temps = list(np.ravel(results))
   
    # jsonify results
    return jsonify(ly_temps=ly_temps)


In [ ]:
# Start and Start/End Routes
@app.route("/api/v1.0/min_avg_max/<start>")
def start_date():
    print("Request for 'Start Date'page...")

    #Create session link to DB
    session=Session(engine)
    
    # When given start date only, calculate TMIN, TAVG, and TMAX 
    # for all dates greater than and equal to start date
    # Referenced 10.3 3 "Analysis" for analyzing average prices and 10.3 #2 for Datetimes
    # Referenced 
    sel = [func.min(Measurement.tobs),
           func.avg(Measurement.tobs),
           func.max(Measurement.tobs)]
    
        # Calculate TMIN, TAVG, TMAX for dates greater than and equal to the start
        tob_queries = session.query(*sel).\
            filter(Measurement.date >= start).all()
        
        # Convert List of Tuples into a normal list
        temp_data = list(np.ravel(tob_queries))
        return jsonify(temps)

    # Calculate TMIN, TAVG, TMAX for dates between the start and end date inclusive
    # Use flask jsonify to convert your API data into valid JSON Response object
    # Referenced 10.3 #3 "Analysis" and 10.3 #2 for Datetimes
    tob_queries = session.query(*sel).\
        filter(Measurement.date >= start).\
        filter(Measurement.date <= end).all()
  
    # Convert List of Tuples into a normal List
    temp_data = list(np.ravel(tob_queries))
    return jsonify(temp_data)
    
if __name__ == '__main__':
    app.run()